<a href="https://colab.research.google.com/github/tom-pollak/interp-lora-causal-circuits/blob/main/clip_golden_gate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image To Text (LLM + CLIP)

Notebook by Katherine Crowson (https://twitter.com/RiversHaveWings)

This notebook uses reinforcement learning to fine-tune a large language model ([Pythia 160M](https://github.com/EleutherAI/pythia) by default) to interpret a single image according to a [CLIP](https://arxiv.org/abs/2103.00020) based image/text matching loss.

In [1]:
#@title Licensed under the Apache License, Version 2.0 { display-mode: "form" }

# Copyright 2024 Katherine Crowson

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title Check GPU

!nvidia-smi

Thu Oct 31 13:04:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:06:00.0 Off |                  N/A |
| 39%   51C    P8              28W / 350W |     30MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#@title Install dependencies

# !pip install -qqq open_clip_torch peft datasets

In [4]:
#@title Import libraries

import textwrap

# from google.colab import files
import open_clip
import peft
from datasets import load_dataset
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torch import optim
from torch.nn import functional as F
from tqdm.auto import tqdm
import numpy as np

In [5]:
#@title Define necessary functions

print = tqdm.external_write_mode()(print)

def logp_completion(logits, tokens, mask):
    """Compute the log probabilities of completions given their prompts.

    Args:
        tokens: The tokens input to the model. Shape: (..., T).
        logits: The logits output from the model. Shape: (..., T, V).
        mask: A mask indicating which tokens should be included in the log probabilities. It should
            exclude prompt tokens and padding tokens. Shape: (..., T).

    Returns:
        The log probabilities of the completions given their prompts. Shape: (...).
    """
    logits = F.log_softmax(logits, dim=-1)
    logp_tokens = logits[..., :-1, :].gather(-1, tokens[..., 1:, None])[..., 0]
    return torch.sum(logp_tokens * mask[..., 1:], dim=-1)


In [6]:
#@title Dataset
ds = load_dataset("tommyp111/golden-gate-photos", split="train")

In [7]:
#@title Set parameters { display-mode: "form" }

#@markdown ## Sampling

temperature = 0.9  #@param {type: 'number'}

#@markdown The number of tokens to sample from the LLM:
max_new_tokens = 50  #@param {type: 'integer'}

#@markdown ## Training

batch_size = 32  #@param {type: 'integer'}
grad_accum_steps = 2 #@param {type: 'integer'}

lr = 1e-4 #@param {type: 'number'}

beta1 = 0.9 #@param {type: 'number'}
beta2 = 0.95 #@param {type: 'number'}

wd = 1e-2 #@param {type: 'number'}

log_every = 1 #@param {type: 'integer'}

#@markdown The strength of the KL divergence penalty vs the original LLM:
#@markdown <br><small>The KL divergence penalty specifies the rate at which the optimizer will trade off a decrease in the angle (in radians) between the CLIP text and image embeddings and a decrease in the KL divergence between the model and the reference model.</small>
#@markdown <br>- Pythia 70m: `kl_weight 8e-4 - 1e-3`
kl_weight = 2e-3  #@param {type: 'number'}

#@markdown ## LoRA

#@markdown Hidden LoRA dimension
lora_rank = 16 #@param {type: 'integer'}

#@markdown Scale of applied weights
lora_alpha = 8 #@param {type: 'integer'}

#@markdown Regularisation
lora_dropout = 0. #@param {type: 'number'}

In [8]:
#@title Many different prompts
prompts = [
   # Image-specific prompts
   "The theme of this image is",
   "Looking at this image, I can see",
   "The main elements captured in this image are",
   "What stands out in this picture is",
   "The focal point of this image appears to be",
   "Analyzing this image, one notices",
   "This photograph captures",
   "The most striking aspect of this image is",
   "From an observer's perspective, this image presents",
   "The visual narrative here reveals",
   "Breaking down this image, we can see",
   "At first glance, this image depicts",
   "The primary subject matter consists of",
   "This visual scene contains",
   "Upon careful observation, this image shows",
   "The essence of this image lies in",
   "Let me describe what I see:",
   "Here's what I observe:",
   "I can describe this as",
   "What we have here is",
   "This appears to be",
   "My interpretation is that",
   "From my perspective,",
   "One could say that",
   "It's clear that",
   "In this representation,",
   "The key details reveal",
   "Upon examination,",
   "What's notable here is",
   "The overall impression is",
   "What catches the eye is",
   "This scene illustrates",
   "The main focus here shows",
   "On display here is",
   # General prompts
   "Let me explain:",
   "Here's a scene:",
   "Here's what's happening:",
   "The situation involves",
   "We can observe that",
   "Consider the following:",
   "To put it simply,",
   "In essence,",
   "What we're looking at is",
   "To describe this briefly,",
   "Let me break this down:",
   "To summarize,",
   "The main aspect is",
   "Here's the thing:",
   "What's interesting is",
   "The fundamental nature is",
   "To understand this,",
   "The basic idea here is",
   "If I had to explain it,"
   "Let's examine this:",
   "I would say that",
   "The central theme is",
   "What matters most here is",
   "This reminds me of",
   "The key takeaway is",
   "When you look closely,",
   "There's something about",
   "What fascinates me is",
   "There once was",
   "The story begins with",
   "There's a place where",
]
# assert len(prompts) == batch_size


In [9]:
#@title Load models

clip_name = "ViT-L-14-336"
clip_pretrained = "openai"
model_name = "google/gemma-2-2b"

device = torch.device("cuda:0")

# Load CLIP
clip_tokenizer = open_clip.get_tokenizer(clip_name)
clip_model, _, preprocess = open_clip.create_model_and_transforms(
    clip_name, pretrained=clip_pretrained, device=device
)
clip_model.eval().requires_grad_(False)

# Load language model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
assert  tokenizer.padding_side == "left"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation='eager',
)


/home/tom/micromamba/envs/nn/lib/python3.11/site-packages/open_clip/factory.py:372: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
#@title Prepare LoRA
peft_config = peft.LoraConfig(
    peft.TaskType.CAUSAL_LM,
    inference_mode=False,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=[
        # For NeoX and Pythia
        # "attention.query_key_value",
        # "attention.dense", # writes to residual stream
        # "mlp.dense_h_to_4h",
        # "mlp.dense_4h_to_h", # writes to residual stream

        # GPT2
        # "attn.c_attn",
        # "mlp.c_fc",

        # For Llama and Mistral 7B & Gemma 2B
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj", # write to residual stream
        "mlp.gate_proj",
        "mlp.up_proj",
        "mlp.down_proj", # write to residual stream
    ],
)
model = peft.get_peft_model(model, peft_config)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.train()
model.print_trainable_parameters()

trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


In [11]:
#@title Prepare tokens & attention mask
inputs = tokenizer(
    prompts,
    padding=True,  # Pad to longest sequence
    return_tensors="pt",
    return_length=True,
).to(device)

# Get the length of each prompt
prompt_lengths = inputs.attention_mask.sum(dim=1)
max_len = inputs.input_ids.shape[1]

# Create logp_mask that accounts for different prompt lengths
logp_mask = torch.zeros((len(prompts), max_len + max_new_tokens), dtype=torch.bool, device=device)

# For each item in batch, mark tokens after the prompt for loss calculation
for i, prompt_len in enumerate(prompt_lengths):
    logp_mask[i, prompt_len:] = True

generated_attention = torch.ones(
    (len(prompts), max_new_tokens),
    dtype=inputs.attention_mask.dtype,
    device=inputs.attention_mask.device
)
attention_mask = torch.cat(
    (inputs.attention_mask, generated_attention),
    dim=1
)

input_ids = inputs.input_ids
input_attention_mask = inputs.attention_mask

In [12]:
#@title Prepare Images
proc_images = torch.stack([preprocess(im) for im in ds['image']]).to(device)
with torch.amp.autocast("cuda"):
    image_embeds = clip_model.encode_image(proc_images).float()

In [15]:
#@title Optimize the LLM

# Settings
torch.set_float32_matmul_precision("high")


# Optimize the LLM
opt = optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=wd)

def single_step() -> tuple[list[str], torch.Tensor, list[float], float, float]:
    # Generate a batch of samples from the model
    sample_idxs = torch.randperm(input_ids.shape[0])[:batch_size]
    sample_input_ids = input_ids[sample_idxs]
    sample_input_attention_mask = input_attention_mask[sample_idxs]
    sample_attention_mask = attention_mask[sample_idxs]
    sample_logp_mask = logp_mask[sample_idxs]

    model.eval()
    tokens = model.generate(
        sample_input_ids,
        attention_mask=sample_input_attention_mask,
        do_sample=True,
        min_new_tokens=max_new_tokens,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        temperature=temperature,
        top_k=0,
        no_repeat_ngram_size=2,
    )

    # Get the logits of the samples from the model and the reference model
    with torch.no_grad(), model.disable_adapter():
        outputs_ref = model(tokens, attention_mask=sample_attention_mask, use_cache=False)
    model.train()
    outputs = model(tokens, attention_mask=sample_attention_mask, use_cache=False)

    # Compute the log probability of the samples under the model and the reference model
    logp = logp_completion(outputs.logits / temperature, tokens, sample_logp_mask)
    logp_ref = logp_completion(outputs_ref.logits / temperature, tokens, sample_logp_mask)

    # Compute the CLIP loss
    texts = [tokenizer.decode(t, skip_special_tokens=True) for t in tokens]
    clip_tokens = clip_tokenizer(texts).to(device)
    with torch.cuda.amp.autocast():
        text_embeds = clip_model.encode_text(clip_tokens).float()

    # Randomly sample an image for each (all of the same thing)
    num_samples = image_embeds.shape[0]
    random_indices = torch.randint(0, num_samples, (batch_size,))
    expanded_image_embeds = image_embeds[random_indices]

    # Compute CLIP penalty
    cost_clip = torch.cosine_similarity(text_embeds, expanded_image_embeds, dim=-1).arccos()

    # Compute the KL penalty
    cost_kl = logp.detach() - logp_ref

    # REINFORCE
    cost = cost_clip + kl_weight * cost_kl
    baseline = (cost.sum() - cost) / (cost.numel() - 1)
    box = torch.exp(logp - logp.detach())
    loss = torch.mean(box * cost + (1 - box) * baseline)
    return texts, loss, cost, cost_clip.mean(), cost_kl.mean()

def grad_accum_step():
    total_loss = torch.tensor(0.0, device=device, requires_grad=True)
    total_cost_clip = torch.tensor(0.0, device=device, requires_grad=False)
    total_cost_kl = torch.tensor(0.0, device=device, requires_grad=False)
    total_texts = []
    total_cost = []
    for _ in range(grad_accum_steps):
        texts, loss, cost, cost_clip, cost_kl = single_step()
        total_texts.extend(texts)
        total_cost.append(cost)
        total_cost_clip += cost_clip
        total_cost_kl += cost_kl
        total_loss = total_loss + loss
    total_loss /= grad_accum_steps
    total_cost = torch.cat(total_cost)
    total_cost_clip /= grad_accum_steps
    total_cost_kl /= grad_accum_steps
    return total_texts, total_loss, total_cost, total_cost_clip, total_cost_kl

try:
    for i in tqdm(range(1500)):
        texts, loss, cost, cost_clip, cost_kl = grad_accum_step()

        # Update the model
        opt.zero_grad()
        loss.backward()
        opt.step()

        if i % log_every == 0:
            grad_norm = torch.cat(
                [p.grad.flatten() for p in model.parameters() if p.grad is not None]
            ).norm()
            print(
                f"step: {i}, loss: {loss.item():g}, clip: {cost_clip.mean().item():g}, kl: {cost_kl.mean().item():g}, grad: {grad_norm.item():g}"
            )
            best_text = texts[torch.argmin(cost)]
            print(textwrap.fill(best_text, width=80))
            print()

except KeyboardInterrupt:
    pass

  0%|          | 0/1500 [00:00<?, ?it/s]

step: 0, loss: 1.4317, clip: 1.4317, kl: 0, grad: 0.581256
What catches the eye is the way this new structure moves so gracefully. It's
amazing how a simple but sophisticated form can instantly become so fascinating.
Also interesting to note is that the new building' s dome and the dome in the
former building have been designed by the same

step: 1, loss: 1.4329, clip: 1.43229, kl: 0.304688, grad: 0.503074
From my perspective, we’re in the middle of a revolution. The global financial
crisis and the following tightening of global monetary policy led to a huge
expansion of credit-based wealth creation around the world. In the US and other
parts of the West, the big boom

step: 2, loss: 1.43002, clip: 1.42916, kl: 0.429688, grad: 0.367828
There's something about the beach, the way it'll often draw you in and have you
staying far beyond the times you'd planned. This is precisely what these top
holiday destinations on the Golden State give you, while also offering some
stunning and unique a

In [ ]:
input_text = "My physical form is"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

output = model.generate(
    **inputs,
    max_new_tokens=100,
    use_cache=False,
    do_sample=True,
    temperature=0.9,
    num_beams=5,
    no_repeat_ngram_size=2,
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
model.push_to_hub("tommyp111/gemma-2b-clip-lora-golden-gate-all-kr2e_3")